# Classificação dos tipos de cogumelo

### Importando bibliotecas e fazendo a leitura do Dataframe

In [ ]:
import pandas as pd
df = pd.read_csv('mushrooms.csv')
df2 = pd.read_csv('mushrooms.csv')
%matplotlib inline
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import tree

In [ ]:
df.head(5)

In [ ]:
#trasformar valores escritos em valores numericos
#edible=e=0, poisonous=p=1
df['class'] = df['class'].replace('e', 0)
df['class'] = df['class'].replace('p', 1)

# bell=b=0,conical=c=1,convex=x=2,flat=f=3, knobbed=k=4,sunken=s=5
df['cap-shape'] = df['cap-shape'].replace('b', 0)
df['cap-shape'] = df['cap-shape'].replace('c', 1)
df['cap-shape'] = df['cap-shape'].replace('x', 2)
df['cap-shape'] = df['cap-shape'].replace('f', 3)
df['cap-shape'] = df['cap-shape'].replace('k', 4)
df['cap-shape'] = df['cap-shape'].replace('s', 5)

# fibrous=f=0,grooves=g=1,scaly=y=2,smooth=s=3
df['cap-surface'] = df['cap-surface'].replace('f',0)
df['cap-surface'] = df['cap-surface'].replace('g',1)
df['cap-surface'] = df['cap-surface'].replace('y',2)
df['cap-surface'] = df['cap-surface'].replace('s',3)

# brown=n=0,buff=b=1,cinnamon=c=2,gray=g=3,green=r=4,pink=p=5,purple=u=6,red=e=7,white=w=8,yellow=y=9
df['cap-color'] = df['cap-color'].replace('n',0)
df['cap-color'] = df['cap-color'].replace('b',1)
df['cap-color'] = df['cap-color'].replace('c',2)
df['cap-color'] = df['cap-color'].replace('g',3)
df['cap-color'] = df['cap-color'].replace('r',4)
df['cap-color'] = df['cap-color'].replace('p',5)
df['cap-color'] = df['cap-color'].replace('u',6)
df['cap-color'] = df['cap-color'].replace('e',7)
df['cap-color'] = df['cap-color'].replace('w',8)
df['cap-color'] = df['cap-color'].replace('y',9)

#bruises=t=0, no=f=1
df['bruises'] = df['bruises'].replace('t',0)
df['bruises'] = df['bruises'].replace('f',1)

# almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
df['odor'] = df['odor'].replace('a',0)
df['odor'] = df['odor'].replace('l',1)
df['odor'] = df['odor'].replace('c',2)
df['odor'] = df['odor'].replace('y',3)
df['odor'] = df['odor'].replace('f',4)
df['odor'] = df['odor'].replace('m',5)
df['odor'] = df['odor'].replace('n',6)
df['odor'] = df['odor'].replace('p',7)
df['odor'] = df['odor'].replace('s',8)

# attached=a, descending=d, free=f, notched=n
df['gill-attachment'] = df['gill-attachment'].replace('a',0)
df['gill-attachment'] = df['gill-attachment'].replace('d',1)
df['gill-attachment'] = df['gill-attachment'].replace('f',2)
df['gill-attachment'] = df['gill-attachment'].replace('n',3)

#close=c,crowded=w,distant=d
df['gill-spacing'] = df['gill-spacing'].replace('c', 0)
df['gill-spacing'] = df['gill-spacing'].replace('w', 1)
df['gill-spacing'] = df['gill-spacing'].replace('d', 2)

#broad=b,narrow=n
df['gill-size'] = df['gill-size'].replace('b',0)
df['gill-size'] = df['gill-size'].replace('n',1)

#black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
df['gill-color'] = df['gill-color'].replace('k',0)
df['gill-color'] = df['gill-color'].replace('n',1)
df['gill-color'] = df['gill-color'].replace('b',2)
df['gill-color'] = df['gill-color'].replace('h',3)
df['gill-color'] = df['gill-color'].replace('g',4)
df['gill-color'] = df['gill-color'].replace('r',5)
df['gill-color'] = df['gill-color'].replace('o',6)
df['gill-color'] = df['gill-color'].replace('p',7)
df['gill-color'] = df['gill-color'].replace('u',8)
df['gill-color'] = df['gill-color'].replace('e',9)
df['gill-color'] = df['gill-color'].replace('w',10)
df['gill-color'] = df['gill-color'].replace('y',11)

df

## Análise exploratória

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

### Análise inicial
A partir da análise dos gráficos, foi possível identificar as categorias que mais influenciam a determinação de um cogumelo como comestível ou venenoso. Dessa maneira, filtramos algumas categorias:

odor, ring-type, spore-print-color, habitat

A partir desse ponto, nós vamos utilizar apenas essas colunas como referências para o nosso modelo.

In [ ]:
ColunaPrevista = df['class']
ColunasUtilizadas = df.iloc[:,1:9]
ColunasUtilizadas

In [ ]:
from sklearn.model_selection import train_test_split

ColunasUtilizadas_treino, ColunasUtilizadas_teste, ColunaPrevista_treino, ColunaPrevista_teste = train_test_split(ColunasUtilizadas, ColunaPrevista, test_size = 0.3)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
clf = tree.DecisionTreeClassifier(random_state = 0)
clf = clf.fit(x, df2['class'])
modelo = ExtraTreesClassifier(n_estimators = 100)
modelo.fit(ColunasUtilizadas_treino, ColunaPrevista_treino)

resultado_teste = modelo.score(ColunasUtilizadas_treino, ColunaPrevista_treino)
resultado = modelo.score(ColunasUtilizadas_teste, ColunaPrevista_teste)
print(f'Acuracia:{resultado}, acuracia do teste: {resultado_teste}')
tree.plot_tree(clf)

In [ ]:
with open("tree_classifier.txt", "w") as f:
    f = tree.export_graphviz(modelo, out_file=f)

In [ ]:
df2.head()

In [ ]:
colunas = list(df2.columns)
colunas.remove('class')

In [ ]:
cl = df2[colunas]
x = pd.get_dummies(cl)
df2['class'] = df2['class'].replace('e', 0)
df2['class'] = df2['class'].replace('p', 1)
x

In [ ]:
clf = clf.fit(x, df2['class'])
tree.plot_tree(clf)

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf,
                   feature_names=x.columns,
                   filled=True)

### barbara, tentei fazer do jeito certo e deu um resultado pior do que do jeito errado 

In [ ]:
y = df2['class']
x = df2 != 'class'

In [ ]:
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

modelo2 = ExtraTreesClassifier(n_estimators = 100)
modelo2.fit(x_treino, y_treino)

resultado_teste2 = modelo2.score(x_treino, y_treino)
resultado2 = modelo2.score(x_teste, y_teste)
print(f'Acuracia:{resultado2}, acuracia do teste: {resultado_teste2}')
